# initialisation
## importation des libraires

In [1]:
from bs4 import BeautifulSoup
import requests
import re

import time
import math

import pandas as pd

## création variable
* nom de fichier de l'export
* dictionnaire vide pour les prochains résultats

In [2]:
nom_export = "exports/export_0-500.xlsx"

data_result = {
    "nom" : [],
    "EAN" : [],
    "url" : [],
    "marque" : [],
    "modele" : []
}

## récuperation de la base de données agrizone 
présent dans un export excel

In [3]:
doc_export = pd.read_excel(nom_export)

# fonctions

## fonction de scraping de la fiche produit
qui permet de faire le scrap et de séléctionner 

Ici j'ai eu un problème et du créer une autre fonction :
* une fonction pour enlever les balise html d'une suite d'un string et ne renvoyer que le contenu texte.

In [5]:
def enlever_balise(text : str):
    
    
    #on regarde si c'est une liste
    if(text.find('<li') != -1):
        is_liste = True
    else:
        is_liste = False
        
        

    #si c'est une liste, on init en liste sinon en str (pour la 2eme value)
    if (is_liste):
        rep = ["",[]]
        item = ''
    else:
        rep = ["",""]
    
    
    open_balise, nom_donne = False, False
    
    for i in text:
        if(open_balise):
            if(i == ">"):
                open_balise = False
                if (is_liste and len(item) > 2 ):
                    rep[1].append(item)
                    item = ''
        else:
            if(i == "<"):
                open_balise = True
            if(not(open_balise)):
                if(nom_donne):
                    if (is_liste):
                        item += str(i)
                    else:
                        rep[1] += str(i)
                else:
                    rep[0] += str(i)
            else:
                #si le nom n'est pas encore présent
                if(len(rep[0]) != 0 and not(nom_donne)):
                    nom_donne = True
    
    return (rep)

def scrap_fiche_produit(url):
    nom, marque, produit = None, None, None
    
    req = requests.get(url)
    soup = BeautifulSoup(req.text)
        
    #recup le nom
    for i in soup.find_all('span'):
        if(i.get('itemprop') == 'name'):
            nom = str(i)[22:-7]
    
    #recup les attributs
    for i in soup.find_all('table'):
        for n in range(len(i.find_all('tr'))):
            if (n != 0):
                nom_attrib, txt_attrib = enlever_balise(str(i.find_all('tr')[n]))
                
                if (nom_attrib == "Convient pour la marque de machine/véhicule"):
                    marque = txt_attrib
                elif (nom_attrib == "Convient pour le modèle de machine/véhicule"):
                    produit = txt_attrib
    
    
    return nom, marque, produit

## fonction de recherche de résultat à partir d'un url
la fonction prend un url des pages de recherche Kramp en entrée et renvoie une liste de balise href en html

comme par exemple la page : https://www.kramp.com/shop-fr/fr/search/"la_recherche"

In [4]:
def search_result(url):
    
    rep = []
    
    req = requests.get(url)
    soup = BeautifulSoup(req.text)

    for div in soup.find_all('div'):
        if(div.get('data-intent') == 'contains-product-item-from-search-result'):
            rep.append(str(div)[str(div).find('href'):str(div).find('href')+str(div)[str(div).find('href')+6:].find('"')+7])
            
    return rep

## recherche est la fonction global 
qui appele toutes les autres qui sont liés a la récupération des informations sur Kramp

In [6]:
#ouverture du document
doc_export = pd.read_excel(nom_export)

#init des bases de recherche
site_base = "https://www.kramp.com/"
search_base = site_base + "shop-fr/fr/search/"


#recherche demande le code ean d'un produit et renvoie le nom, code_ean, marque, modele et l'url.
def recherche(code_ean : str):
       
    #recherche des résultat
    url = search_base + str(code_ean)
    hrefs = search_result(url)
    
    if (len(hrefs) == 1):
    #la recherche a donné un résultat
        
        #on créé l'url du résultat rechercher avant
        url = site_base[:-1] + hrefs[0][6:-1]
        
        #on scrap la fiche produit de l'url
        nom, marque, modele = scrap_fiche_produit(url)
        
        
    else:
    #la recherche n'a pas donné qu'un résultat
        url, nom, marque, modele = None, None, None, None
    
    return nom, code_ean, marque, modele, url

# fonction qui servent à classé les données

## enregister est la fonction qui enregistre
elle permet d'enregistrer dans le dictionnaire
## to_excel est la fonction qui enregistre en externe
elle permet de ne pas perdre les résultats (le dictionnaire) si la fonction crash ou quoi que se soit, et vide le dictionnaire

In [7]:
#la def enregistrer demande le nom, code_ean, marque, modele et l'url et l'enregistre dans le dictionnaire data_result et ne renvoie rien
def enregistrer(nom, code_ean, marque, modele, url):
    data_result["nom"].append(nom)
    data_result["EAN"].append(code_ean)
    data_result["marque"].append(marque)
    data_result["modele"].append(modele)
    data_result["url"].append(url)    
    pass

"""
la def to_excel demande les indexs de début et de fin et enregistre le contenu du dictionnaire data_result dans un fichier nommé comme tel :
    -->  result_"index de début"-"index de fin".xlsx  <--
elle ne renvoie rien
"""
def to_excel(index_start: int, index_end: int):
    df = pd.DataFrame(data_result)
    df.to_excel("results/result_" + str(index_start) + "-" + str(index_end) + ".xlsx")
    
    print('rec')
        
    data_result["nom"] = []
    data_result["EAN"] = []
    data_result["marque"] = []
    data_result["modele"] = []
    data_result["url"] = []
    
    pass

# main du programme

la boucle qui permet de faire fonctionner automatiquement le programme 

In [8]:
time_start = time.time()

index_start = 0

for index in range(len(doc_export['Code EAN produit'])):
    
    ean = doc_export['Code EAN produit'][index]
    nom, code_ean, marque, modele, url = recherche(ean)
    enregistrer(nom, code_ean, marque, modele, url)
    
    if ((index % 20 ) == 0):
        print(round(index / len(doc_export['Code EAN produit']) * 100),"%")
        pass
        
    if (((index % 250 ) == 0 and index is not 0) or index == len(doc_export['Code EAN produit'])):
        to_excel(index_start,index)
        index_start = index + 1

time_end = time.time()

nom   : Dent de herse Ø9 p/Amazone
code  : 8716106344109
marqu : Amazone 
model : None
url   : https://www.kramp.com/shop-fr/fr/p/dent-de-herse-%C3%B89-p-amazone--952459N
_____
nom   : Soc de rasette G p/Kverneland
code  : 8716106171224
marqu : Kverneland 
model : None
url   : https://www.kramp.com/shop-fr/fr/p/soc-de-rasette-g-p-kverneland--KK066873N
_____
nom   : Pointe MulchMix XXL carb. p/Horsch
code  : 8716106710232
marqu : Horsch 
model : MulchMix XXL 
url   : https://www.kramp.com/shop-fr/fr/p/pointe-mulchmix-xxl-carb.-p-horsch--00311069KR
_____
nom   : Pointe réver. 3000 D p/Fenet (FWS)
code  : 8716106714339
marqu : Fenet 
model : None
url   : https://www.kramp.com/shop-fr/fr/p/pointe-r%C3%A9ver.-3000-d-p-fenet-fws--010900104N
_____
nom   : Pointe carb. D. p/Naud
code  : 8716106305650
marqu : Naud 
model : None
url   : https://www.kramp.com/shop-fr/fr/p/pointe-carb.-d.-p-naud--031234DCKR
_____
nom   : Soc RD universel HC gauche
code  : 8719607076890
marqu : Väderstad 
model : N

# externe
fonction pour voir combien de temps l'execution à pris

hh:mm:ss

In [ ]:
code_ean = "8716106944545"

req = requests.get('https://www.kramp.com/shop-fr/fr/search/8' + code_ean)
soup = BeautifulSoup(req.text)

In [9]:
print(str(math.floor((time_end - time_start) / 3600 )) + ':' +
      str(math.floor(((time_end - time_start) / 60 ) % 60)) + ':' +
      str(math.floor((time_end - time_start) % 60 )))

0:14:21
